In [1]:
"""
@Title: 金融中的线性问题
@Author(s): Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime: 2023-10-18 21:53:24
@Description: 
"""

'\n@Title: 金融中的线性问题\n@Author(s): Stephen CUI\n@LastEditor(s): Stephen CUI\n@CreatedTime: 2023-10-18 21:53:24\n@Description: \n'